## 实现一个带参数的缓存装饰器，允许指定过期时间

In [1]:
from functools import wraps

def cache(func):
    data = {}

    @wraps(func)
    def wrapper(*args):
        if args in data:
            print("in cache")
            return data[args]
        else:
            print("not in cache")
            res = func(*args)
            data[args] = res
            return res

    return wrapper


@cache
def add(x, y):
    return x + y


print(add(1, 2))
print(add(1, 2))
print(add(3, 4))

not in cache
3
in cache
3
not in cache
7


In [2]:
add(y=2, x=1)

TypeError: wrapper() got an unexpected keyword argument 'y'

In [ ]:
def cache(fn):
    # 存储缓存
    c = {}
    
    @wraps(fn)
    def wrap(*args, **kwargs):
        # key ? args, kwargs
        # 超时检测
        ret = fn(*args, **kwargs)
        c[key] = ret
        return ret

参数构造： "x=1&y=2"

kwargs 可以获取到key

args 的key 如何获取？

In [3]:
import inspect

In [4]:
def add(x, y):
    pass

In [5]:
sig = inspect.signature(add)

In [6]:
sig

<Signature (x, y)>

In [7]:
sig.parameters

mappingproxy({'x': <Parameter "x">, 'y': <Parameter "y">})

In [8]:
sig.parameters.items()

odict_items([('x', <Parameter "x">), ('y', <Parameter "y">)])

In [9]:
sig.parameters.keys()

odict_keys(['x', 'y'])

In [14]:
list(sig.parameters.keys())[0]

'x'

In [11]:
for k in sig.parameters.keys():
    print(k)

x
y


In [31]:
def cache(fn):
    # 存储缓存
    c = {}
    
    @wraps(fn)
    def wrap(*args, **kwargs):
        # key ? args, kwargs
        print(args)
        print(kwargs)
        key = []
        
        params = inspect.signature(fn).parameters
        # args
        for i, arg in enumerate(args):
            # args 是 位置参数，所以他的顺序和我们的params顺序 应该是一致的
            # name 是我们的key
            name = list(params.keys())[i]
            key.append((name, arg))
        
        # kwrgs
#         for k, v in kwargs.items():
#             key.append((k, v))
            
        key.extend(kwargs.items())
        
        key.sort(key=lambda x: x[0])
        
        key = '&'.join([f'{name}={arg}' for name, arg in key])
        
        print(key)
        
        # 命中检测
        if key in c.keys():
            # 超时检测
            print(f'命中缓存：{key}: {args}, {kwargs}')
            return c[key]
        
        ret = fn(*args, **kwargs)
        c[key] = ret
        return ret
    return wrap

In [ ]:
add(1, 2) # key => [(x, 1), (y, 2)] 

In [21]:
@cache
def add(x, y):
    return x + y

In [22]:
add(1, 2)

x=1&y=2


3

In [23]:
add(1, 2)

x=1&y=2
命中缓存：x=1&y=2: (1, 2), {}


3

In [24]:
add(3, 4)

x=3&y=4


7

In [25]:
add(3,4)

x=3&y=4
命中缓存：x=3&y=4: (3, 4), {}


7

In [26]:
add(y=4, x=3)

x=3&y=4
命中缓存：x=3&y=4: (), {'y': 4, 'x': 3}


7

In [27]:
@cache
def add(x, y=3):
    return x + y

In [28]:
add(1, 3)

x=1&y=3


4

In [29]:
add(1, 3)

x=1&y=3
命中缓存：x=1&y=3: (1, 3), {}


4

In [30]:
add(1)

x=1


4

In [32]:
@cache
def fn(a, b=1):
    pass

In [33]:
fn(1)

(1,)
{}
a=1


In [34]:
fn(a=1)

()
{'a': 1}
a=1
命中缓存：a=1: (), {'a': 1}


In [44]:
def cache(fn):
    # 存储缓存
    c = {}
    
    @wraps(fn)
    def wrap(*args, **kwargs):
        # key ? args, kwargs
        print(args)
        print(kwargs)
        key = []
        
        names = set()
        
        params = inspect.signature(fn).parameters
        # args
        for i, arg in enumerate(args):
            # args 是 位置参数，所以他的顺序和我们的params顺序 应该是一致的
            # name 是我们的key
            name = list(params.keys())[i]
            key.append((name, arg))
            
            names.add(name)
        
        # kwrgs
#         for k, v in kwargs.items():
#             key.append((k, v))
            
        key.extend(kwargs.items())
        
        names.update(kwargs.keys())
        
        for k, v in params.items():
            # 如何获取到默认参数
            # 如果定义函数的时候，有默认参数，在传参的时候没有传这个默认参数，我们就拿不到它
            # params是整个函数的参数列表
            # params 里 不在 names 里的 就是默认参数
            if k not in names:
                key.append((k, v.default))
        
        key.sort(key=lambda x: x[0])
        
        key = '&'.join([f'{name}={arg}' for name, arg in key])
        
        print(key)
        
        # 命中检测
        if key in c.keys():
            # 超时检测
            print(f'命中缓存：{key}: {args}, {kwargs}')
            return c[key]
        
        ret = fn(*args, **kwargs)
        c[key] = ret
        return ret
    return wrap

In [35]:
def fn(a, b=1):
    pass

In [36]:
sig = inspect.signature(fn)

In [37]:
sig.parameters

mappingproxy({'a': <Parameter "a">, 'b': <Parameter "b=1">})

In [41]:
ret = list(sig.parameters.values())[1]

In [42]:
ret.default

1

In [43]:
list(sig.parameters.values())[0].default

inspect._empty

In [45]:
@cache
def add(a, b=2):
    return a + b

In [46]:
add(1, 2)

(1, 2)
{}
a=1&b=2


3

In [47]:
add(b=2, a=1)

()
{'b': 2, 'a': 1}
a=1&b=2
命中缓存：a=1&b=2: (), {'b': 2, 'a': 1}


3

In [48]:
add(1)

(1,)
{}
a=1&b=2
命中缓存：a=1&b=2: (1,), {}


3

In [61]:
import datetime 

def cache(exp=0, t):
    # exp=0 没有超时时间
    def _cache(fn):
        # 存储缓存
        c = {}
        
        def lru():
            pass
        
        def fifo():
            pass

        @wraps(fn)
        def wrap(*args, **kwargs):
            # key ? args, kwargs
            print(args)
            print(kwargs)
            key = []

            names = set()

            params = inspect.signature(fn).parameters
            # args
            for i, arg in enumerate(args):
                # args 是 位置参数，所以他的顺序和我们的params顺序 应该是一致的
                # name 是我们的key
                name = list(params.keys())[i]
                key.append((name, arg))

                names.add(name)

            # kwrgs
    #         for k, v in kwargs.items():
    #             key.append((k, v))

            key.extend(kwargs.items())

            names.update(kwargs.keys())

            for k, v in params.items():
                # 如何获取到默认参数
                # 如果定义函数的时候，有默认参数，在传参的时候没有传这个默认参数，我们就拿不到它
                # params是整个函数的参数列表
                # params 里 不在 names 里的 就是默认参数
                if k not in names:
                    key.append((k, v.default))

            key.sort(key=lambda x: x[0])

            key = '&'.join([f'{name}={arg}' for name, arg in key])

            print(key)

            # 命中检测
            if key in c.keys():
                # 超时检测
                # 如何判断 一组key 在什么时候执行过呢？
                # key &ts=1231231231
                # key = (key, timestamp): values
                ret, timestamp = c[key]
                if exp == 0 or datetime.datetime.now().timestamp() - timestamp < exp:
                    print(f'命中缓存：{key}: {args}, {kwargs}')
                    return c[key]

            ret = fn(*args, **kwargs)
            print('缓存未命中')
            c[key] = (ret, datetime.datetime.now().timestamp())
            return ret
        return wrap
    return _cache

In [50]:
import datetime

In [53]:
datetime.datetime.now().timestamp()

1544921875.402381

In [62]:
@cache(5)
def add(a, b=2):
    return a + b

In [63]:
add(1, 2)

(1, 2)
{}
a=1&b=2
缓存未命中


3

In [64]:
add(2, 1)

(2, 1)
{}
a=2&b=1
缓存未命中


3

In [65]:
add(a=1,b=2)

()
{'a': 1, 'b': 2}
a=1&b=2
命中缓存：a=1&b=2: (), {'a': 1, 'b': 2}


(3, 1544922062.303333)

In [66]:
add(a=1,b=2)

()
{'a': 1, 'b': 2}
a=1&b=2
缓存未命中


3

In [67]:
1 / 0 

ZeroDivisionError: division by zero